In [133]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [150]:
def init_solution(k,n):
    archive = np.random.uniform(-1,1,size=(k*n)).reshape(k,n)
    return archive

In [187]:
def calculate_fitness(nn_model,archive,summary_writter=None):
    fitness_solution = np.zeros(archive.shape[0])
    for (index,candidate) in enumerate(archive):
        result = nn_model.compute_error(candidate)
#         print result
        fitness_solution[index] = result
    sorted_idx = np.argsort(fitness_solution)
    sorted_archive = np.zeros(archive.shape)
    for (index,item) in enumerate(sorted_idx):
        sorted_archive[item] = archive[index]
    summary_writter.append(fitness_solution.min())
    return sorted_archive

In [102]:
def calculate_weights(q,shape):
    weights = np.zeros([shape[0],1])
    # qk = 0.1
    co_efficient = 1 / (0.1 * np.sqrt(2*np.pi))
    for index in np.arange(shape[0]):
        exponent = np.square(index-1) / (2*np.square(q*shape[0]))
        weights[index] = co_efficient * np.exp(-exponent)
    return weights

In [103]:
def compute_standard_deviation(i,l,archive,epsilon):
    __doc__ = " compute standard deviation with i, l, archive and epsilon"
    #Constant sd
    sd = 0.01
    co_efficient = epsilon / (archive.shape[0])
    sum_sd = np.multiply(np.sum(archive[l]-archive[l][i]),epsilon)
    if(sum_sd <= 0):
        sum_sd = sd
    return sum_sd

In [104]:
def choose_pdf(archive_shape,weights):
    sum_weights = np.sum(weights)
    temp = 0
    l = 0
    pro_r = np.random.uniform(0.0,1.0)
    for (index,weight) in enumerate(weights):
        temp = temp + weight/sum_weights
        if(temp > pro_r):
            l = index
    return l

In [168]:
def sampling_more(archive,weights,epsilon):
    try:
        sigma = 0.0
        mu = 0.0
        pdf = 0
        next_archive = np.zeros(archive.shape)
        for index in np.arange(archive.shape[0]):
            i_pdf = choose_pdf(archive.shape,weights)
            for item in np.arange(archive.shape[1]):
                sigma = compute_standard_deviation(item,i_pdf,archive,epsilon)
                mu = archive[pdf][item]
    #             print sigma,mu
                next_archive[index][item] = np.random.normal(mu,sigma)
    except Exception as e:
        print "Sampling more"
        print e
    return next_archive

In [203]:
class NeuralNetwork:
    def __init__(self,hidden_node):
        print "Initilization"
        self.hidden_node = hidden_node
    def fit(self,X_training,y_training):
        self.X_training = X_training
        self.y_training = y_training
    def activation(self,X,weights):
#         N = weights.shape[0]
#         weights_reshape = weights.reshape(N,1)
#         print weights.shape
#         print X.shape
        return self.sigmoid(np.dot(X,weights))
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    def compute_error(self,weights):
        predict = self.activation(self.X_training,weights)
        return np.sqrt(mean_squared_error(predict,self.y_training))

In [232]:
def AntGradient(k,n,nn_model,summary_writter=None):
    #Initialize parameter for CACO
    number_of_layers = 3
    number_of_weights = n
    epsilon = 0.85
    nn_shape = [4,1]
    q = 0.08 
    max_iteration = 100
    error_criteria = 0.09
    const_sd = 0.1
    
    count_dup = 3
    best_error = np.inf
    archive = init_solution(k,n)
    stop_condition = False
    duplicate = 0
    best_error = np.Inf
    for i in np.arange(max_iteration):
        try:
            sorted_archive = calculate_fitness(nn_model,archive,summary_writter)
#             if(is_accepted(fitness_solution[0])):
# #                 break
            if((summary_writter[-1]-best_error)<=1E-7):
                duplicate+=1
                if(duplicate==count_dup):
                    break
                    print summary_writter[-1]
            else:
                best_error = summary_writter[-1]
            weights = calculate_weights(q,archive.shape)
            archive = sampling_more(sorted_archive,weights,epsilon)
        
        except Exception as e:
            print e,i
            break
    return archive

In [234]:
file_data = np.load("data.npz")
X_training = file_data["X_train"]
y_training = file_data["y_train"].reshape(file_data["y_train"].shape[0],1).astype("float32")
X_test = file_data["X_test"]
y_test = file_data["y_test"].reshape(file_data["y_test"].shape[0],1).astype("float32")

In [235]:
nn_model = NeuralNetwork(15)
nn_model.fit(X_training,y_training)

Initilization


In [236]:
summary_writer = []
weights = AntGradient(100,4,nn_model,summary_writer)
summary_writer

[0.3673113571530936, 0.32669583118349133, 0.28737316365006005]